# Demo

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 124.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [ ]:
!pip install scikit-learn
!pip install nltk
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [ ]:
!git clone https://github.com/lenguyen2592004/fake-news-detection-using-BERT.git

Cloning into 'fakenewsdetection'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 33 (delta 14), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (33/33), 12.23 KiB | 963.00 KiB/s, done.


In [ ]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from tokenizers import Tokenizer
from wordcloud import WordCloud, STOPWORDS
import gc
nltk.download('punkt')
nltk.download('stopwords')
import pickle
model = pickle.load(open("/content/fakenewsdetection/fakenews.sav", 'rb'))
import gradio as gr
import torch
import time
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score

# paramters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def wordpre(text):
    """
    Hàm wordpre có tác dụng chuyển tất cả kí tự trong văn bản thành chữ thường, xoá bỏ các kí tự dấu, đường link, dấu câu, dấu cách, xuống dòng,...
    Những kí tự này không có ý nghĩa đáng kể trong việc phân loại tính chất của câu, việc loại bỏ chúng giúp tập trung vào các từ mang ý nghĩa
    quan trọng, giúp đánh giá câu chính xác hơn, ngoài ra còn giảm kích thước từ điển và cải thiện hiệu suất của mô hình.
    """
    text = text.lower()
    # Chuẩn hóa NKFD và loại bỏ dấu
    text = normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def lower_and_tokenize(data):
    """
    nltk.word_tokenize: tách một đoạn văn bản thành các từ riêng biệt.(token hóa) Việc token hóa giúp
    """
    # Lowercasing and tokenization
    data=data.str.lower()
    data=data.apply(nltk.word_tokenize)
    # Remove stopwords
    stop_words= set(stopwords.words('english'))
    data=data.apply(lambda x: [word for word in x if word not in stop_words])
    return data

def predict(text):
    text=pd.DataFrame([text], columns=["text"])
    text=text["text"]
    text=text.apply(wordpre)
    text=lower_and_tokenize(text)
    nltk.download('wordnet')
    from nltk.corpus import wordnet as wn
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    text = text.apply(lambda x: ' '.join(x))
    # Load the BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    # tokenize and encode sequences in the test set
    max_seq_len=5
    batch_size=16
    tokens_text = tokenizer.batch_encode_plus(
        text.tolist(),
        max_length = max_seq_len,
        padding="max_length",
        truncation=True,
        return_token_type_ids=True,
        add_special_tokens = True,
    )
    # for text set
    text_seq = torch.tensor(tokens_text['input_ids'])
    text_mask = torch.tensor(tokens_text['attention_mask'])
    text_y=torch.tensor([0])
    # wrap tensors
    text_data = TensorDataset(text_seq, text_mask,text_y)

    # sampler for sampling the data during testing
    text_sampler = SequentialSampler(text_data)

    # dataLoader for test set
    text_dataloader = DataLoader(text_data, sampler = text_sampler, batch_size=batch_size)
    seed_val=42
    import random
    import numpy
    random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    y_true = []
    y_pred = []
    use=model
    use.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in text_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        with torch.no_grad():
            out = model(input_ids, input_mask,labels=labels)
            loss = out[0]
            logi = out.logits
            logits_tensor = torch.tensor(logi)
        pred = torch.argmax(logits_tensor, dim = 1)
        y_pred.append(pred.flatten())

    result=str()
    if y_pred==[1]:
       result="This may be a fake news."
    else:
       result="This may be a real news."
    return result

demo = gr.Interface(
    fn=predict,
    inputs=[gr.Textbox(label="Text", lines=3)],
    outputs=[gr.Textbox(label="Predict", lines=1)],
)
if __name__ == "__main__":
    demo.launch(share=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b769462ceee08dd232.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
